## 내 집 마련을 위해, 서울 아파트 매물 데이터를 이용하여 구매 전략을 세우고자 한다.

real_estate.csv

|컬럼|정의|type|
|:---|:---|:---:|
|latitude|위도|float64|
|longitude|경도|float64|
|price_per_square_py|평당가|float64|
|py|평수|int64|
|apt_code|시공사 코드(길이5 알파벳 대문자)|object|
|dist_from_station|인근 지하철역과의 거리|float64|  

정답 및 해설 : https://tjd229.tistory.com/22

In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('../content/tjd229/real_estate.csv')

### Q1. 시공사별 아파트 평수의 평균을 구하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 데이터의 첫 번째 행부터 열 번째 행까지의 시공사 코드(apt_code) 리스트를 저장한다.  
첫 번째 행의 데이터는 위도, 경도, 평당가, 평수, 시공사 코드, 인근 지하철역과의 거리가 각각 37.125541, 126.913776, 2860.053787, 24, GEDAE, 1588.406226이다.  
단계 2 : 시공사 코드(apt_code)가 단계 1에서 구한 리스트에 포함되어 있는 데이터를 전부 제거한다.  
단계 3 : 시공사 코드(apt_code)별 평수 평균을 구하고 그 중 가장 큰 값을 구하시오  

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

In [6]:
df1 = df.copy()
_remove_aptcode = df1.head(10).apt_code.tolist()
_remove_aptcode

['GEDAE',
 'IHEAD',
 'FHIBJ',
 'JBCIJ',
 'EDJCI',
 'EHCAD',
 'HEBIA',
 'HEBIA',
 'IIGFI',
 'FHIBJ']

In [13]:
df1_remove = df1.loc[~df1.apt_code.isin(_remove_aptcode)]
df1_remove

,latitude,longitude,price_per_square_py,py,apt_code,dist_from_station
10,37.046400,126.837832,3750.582844,40,GBDGG,1493.140743
11,37.215855,127.127967,1945.709193,32,HCCBB,989.515201
12,37.252653,127.107425,2670.280398,32,IDCIC,1802.915110
13,37.132489,126.784801,2246.292722,40,HCCBB,922.157169
14,37.270844,127.064936,2423.397245,24,EGFBJ,1491.398873
...,...,...,...,...,...,...
770,37.251483,127.073069,2356.504401,63,HHGCB,1416.903407
771,37.078108,126.723392,1884.239692,24,DEACF,967.757381
773,37.288945,127.118062,3072.810919,24,GBDGG,1803.257080
774,37.234408,126.745186,2491.714188,61,EJAIA,1346.821965


In [25]:
round(df1_remove.groupby('apt_code').mean().max().loc['py'],2)

38.63

### Q2. 아파트 매물을 분석하기 위해 군집화를 하려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.


단계 1 : 독립 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~9개 중 K-means Silhouette 를 통해 구하고, 이 중 첫번째로 높은 score를 최적의 K로 설정한다.  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)를 제외한 모든 컬럼  


#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.cluster import KMeans  
from sklearn.metrics import silhouette_score    
random_state = 229  
문제 지시 외 Default 값 사용  


단계 2 : 단계 1에서 최적의 K로 도출한 각 군집의 비율 중, 가장 큰 값을 구하시오  

※ 정답은 올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

### Q3. 아파트 가격을 예측하는 모델을 만드려고 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

단계 1 : 평당가(price_per_square_py)와 평수(py)를 곱하여 실제 가격 변수를 생성한다.  
단계 2 : Train Set과 Test Set을 28/30:2/30 비율로 나눈다.  


#### 필요 라이브러리 함수,클래스 및 설정값 목록  


from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split  
모든 random_state = 229  
문제 지시 외 Default 값 사용  

In [146]:
df1_real = df.copy()

In [147]:
df1_real['rea_val'] = df1_real.price_per_square_py * df1_real.py
df1_real.head()

,latitude,longitude,price_per_square_py,py,apt_code,dist_from_station,rea_val
0,37.125541,126.913776,2860.053787,24,GEDAE,1588.406226,68641.290895
1,37.223169,127.026913,1715.073163,59,IHEAD,986.146800,101189.316628
2,37.239029,126.906303,1780.604630,57,FHIBJ,1492.514512,101494.463921
3,37.274433,126.737771,1508.814336,42,JBCIJ,1621.602866,63370.202096
4,37.082670,127.143634,2933.930488,32,EDJCI,945.034519,93885.775621


In [148]:
df1_real.drop('apt_code',axis=1, inplace=True)
df1_real.tail()

,latitude,longitude,price_per_square_py,py,dist_from_station,rea_val
772,37.161852,126.917406,3564.856667,40,1593.663253,142594.266660
773,37.288945,127.118062,3072.810919,24,1803.257080,73747.462061
774,37.234408,126.745186,2491.714188,61,1346.821965,151994.565484
775,37.294494,127.185185,1746.372819,40,1461.672552,69854.912763
776,37.004345,127.107611,1667.553347,32,1377.100866,53361.707088


In [149]:
from sklearn.linear_model import LinearRegression  
from sklearn.model_selection import train_test_split

👉 데이터셋 분리

일변일 경우에도 리스트로 전달  :  X = df1_real[['latitude']]

In [150]:
# 독립변수 
X = df1_real[['latitude','longitude','price_per_square_py','dist_from_station','py']]

# 종속변수
y = df1_real['rea_val']

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=2/30, random_state=229)

단계 3 : Train Set으로 LinearRegression 모델을 학습하고, Test Set에 적용한다.   
이 때, 절편을 포함한 경우와 포함하지 않은 경우 각각에 대하여 수행한다.  
  
            - 독립 변수(총 5개) : 시공사 코드(apt_code)와 총 가격을 제외한 모든 컬럼  
            - 종속 변수 : 실제 가격  

단계 3에서 학습한 각 모델에 대하여 예측 결과를 아래 정의된 Measure M으로 계산하였을 때 가장 큰 값은?  
$$ M = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y_i - \hat{y_i})^2}\biggr), \quad \hat{y_i}: 예측값, y_i: 실제값 $$

※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.
(정답 예시: 2.29)

👉 (다중) 선형 회기

In [152]:
real_reg_mo = LinearRegression()
real_reg_mo.fit(X_train, y_train)
y_pred = real_reg_mo.predict(X_test)
y_pred

array([ 71537.06357488,  37200.1213293 ,  35494.01255878,  70471.6408111 ,
        66383.60406998,  75463.1405433 ,  78045.00045754,  58905.29454575,
        65828.22622791,  75100.79955153, 106680.98678017,  78651.20939828,
        61362.85360125,  32623.15234517,  61709.96732249,  36039.69474832,
        61505.47616961,  72107.82903833,  45400.45765664,  68439.60425606,
        72205.20891592,  77442.4900336 , 126070.02549163,  71156.01035643,
        76937.07422418,  80012.34903584,  58971.82398002,  36522.59704217,
        54186.30516848,  51968.30500402,  96691.9974597 , 107056.99700545,
        56149.37950605,  40722.00357319, 155673.66616875,  46700.36282707,
        31876.42694448, 100071.39018113,  38681.4786977 ,  79690.50844088,
        28663.71183769,  50830.05911825,  69194.33878661,  50121.74747649,
        64424.35592352,  68546.3237267 ,  96880.62501702,  95966.21995102,
        68226.19396395,  32537.41804298,  30538.15639693,  76144.92179301])

In [153]:
# 절편이 없는 경우 

real_reg_mo_nointer = LinearRegression(fit_intercept=False)
real_reg_mo_nointer.fit(X_train, y_train)
y_pred_nointer= real_reg_mo_nointer.predict(X_test)
y_pred_nointer

array([ 71427.99389131,  37340.94086899,  35672.38701553,  70390.85330768,
        66588.3801655 ,  75637.33514206,  78237.96205798,  58891.31154552,
        65824.19984416,  75062.4501454 , 106588.90904681,  78579.09045147,
        61516.43389792,  32522.31680278,  61548.54917857,  35972.70060157,
        61599.41435747,  72075.78054911,  45387.05849864,  68426.17098315,
        72201.54983459,  77219.27325969, 125996.43040523,  71214.61840311,
        76707.17492305,  79711.06928229,  58740.54046661,  36453.66058087,
        54334.38241459,  51921.43732011,  96589.6611447 , 107070.36789877,
        56117.52136343,  40769.72426759, 155806.03911506,  46767.95139621,
        31790.48903619, 100418.15476958,  38810.68756198,  79864.38542935,
        28337.99968621,  50653.12697544,  69400.8622678 ,  49904.75210801,
        64288.85849315,  68427.96624782,  96788.24991984,  95832.33827934,
        68329.45721146,  32608.21481703,  30683.22262999,  76166.93103478])

In [154]:
# 절편이 없는 겨우, MSE 의 실제 계산
res = (((y_pred_nointer - y_test)**2).mean())
res

19007465.67489541

- coefficient 의 의미

In [155]:
real_reg_mo.coef_

# 'latitude','longitude','price_per_square_py','dist_from_station','py' 에 대한 기울기.

array([-6.47134835e+02,  8.80496226e+02,  3.50347152e+01,  8.10706193e-01,
        2.21214720e+03])

In [156]:
real_reg_mo.intercept_

-166330.05711401108

### 👉 (다중) 회귀모델 평가
1. MAE (Mean Absolute Error) : (실제값과 예측값) 차이의 절대값
2. MSE (Mean SquareEd Error) : 차이의 제곱
3. RMSE (Root Mean Squared Error) : 차이의 제곱의 루트
4. R2 : 결정계수  (1 - SSE/SST)
    
> R2 는 1에 가까울수록, 나머지는 0에 가까울수록 좋음

In [157]:
# 훈련 데이터 평가
real_reg_mo.score(X_train,y_train)

0.953888849310125

In [158]:
# 테스트 데이터 평가
real_reg_mo.score(X_test,y_test)

0.9689589651349588

1. MAE

In [159]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test, y_pred)

# y_test : 실제값
# y_pred : 모델로 예측한 값

3234.468890751362

2. MSE

In [160]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,y_pred)

19007139.276858058

In [161]:
# 절편이 없는 경우
mean_squared_error(y_test,y_pred_nointer)

# 실제 계산 값 46159607.7447345 과 동일

19007465.67489541

3. RMSE

In [162]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred, squared =False)

4359.7177978463305

4. R2 

** 결과 값이 model.score(X_test, y_test) 와 같다. **

In [163]:
from sklearn.metrics import r2_score
r2_score(y_test,y_pred)

0.9689589651349588